In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Atuona,PF,-9.80,-139.03,80.60,73,1,22.46,clear sky,0.0,0
1,1,Oktyabrskiy,RU,54.48,53.47,64.80,87,2,5.55,clear sky,0.0,0
2,2,Bredasdorp,ZA,-34.53,20.04,48.20,66,0,2.24,clear sky,0.0,0
3,3,Cilegon,ID,-6.02,106.05,78.80,83,20,4.70,haze,0.0,0
4,4,Carutapera,BR,-1.20,-46.02,77.54,89,36,5.68,scattered clouds,0.0,0


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rainingWeather = str(input("Do you want it to be raining? (yes/no) "))
snowingWeather = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain (inches)"] == 0 if rainingWeather == "no" else city_data_df["Rain (inches)"] > 0) & 
                                       (city_data_df["Snow (inches)"] == 0 if snowingWeather == "no" else city_data_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Atuona,PF,-9.80,-139.03,80.60,73,1,22.46,clear sky,0.0,0
3,3,Cilegon,ID,-6.02,106.05,78.80,83,20,4.70,haze,0.0,0
4,4,Carutapera,BR,-1.20,-46.02,77.54,89,36,5.68,scattered clouds,0.0,0
6,6,Zhanaozen,KZ,43.30,52.80,80.37,36,0,3.89,clear sky,0.0,0
7,7,Tual,ID,-5.67,132.75,79.07,81,57,22.91,broken clouds,0.0,0
12,12,Eydhafushi,MV,5.10,73.07,83.39,69,92,3.31,overcast clouds,0.0,0
15,15,Baijiantan,CN,45.63,85.18,75.49,30,100,13.98,overcast clouds,0.0,0
16,16,Mitsamiouli,KM,-11.38,43.28,77.52,73,0,4.94,clear sky,0.0,0
19,19,Palmer,US,42.16,-72.33,88.00,45,40,13.87,scattered clouds,0.0,0
23,23,Jamestown,US,42.10,-79.24,75.00,60,1,12.75,clear sky,0.0,0


In [7]:
preferred_cities_df.count()

City_ID                155
City                   155
Country                155
Lat                    155
Lng                    155
Max Temp               155
Humidity               155
Cloudiness             155
Wind Speed             155
Current Description    155
Rain (inches)          155
Snow (inches)          155
dtype: int64

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,80.60,clear sky,-9.80,-139.03,
3,Cilegon,ID,78.80,haze,-6.02,106.05,
4,Carutapera,BR,77.54,scattered clouds,-1.20,-46.02,
6,Zhanaozen,KZ,80.37,clear sky,43.30,52.80,
7,Tual,ID,79.07,broken clouds,-5.67,132.75,
12,Eydhafushi,MV,83.39,overcast clouds,5.10,73.07,
15,Baijiantan,CN,75.49,overcast clouds,45.63,85.18,
16,Mitsamiouli,KM,77.52,clear sky,-11.38,43.28,
19,Palmer,US,88.00,scattered clouds,42.16,-72.33,
23,Jamestown,US,75.00,clear sky,42.10,-79.24,


In [9]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,80.60,clear sky,-9.80,-139.03,Villa Enata
3,Cilegon,ID,78.80,haze,-6.02,106.05,Forbis Hotel
4,Carutapera,BR,77.54,scattered clouds,-1.20,-46.02,Lidera
6,Zhanaozen,KZ,80.37,clear sky,43.30,52.80,K.korabay 28A
7,Tual,ID,79.07,broken clouds,-5.67,132.75,Grand Vilia Hotel
12,Eydhafushi,MV,83.39,overcast clouds,5.10,73.07,Soneva Fushi
15,Baijiantan,CN,75.49,overcast clouds,45.63,85.18,
16,Mitsamiouli,KM,77.52,clear sky,-11.38,43.28,Foyer ADM
19,Palmer,US,88.00,scattered clouds,42.16,-72.33,Wedgewood Motel
23,Jamestown,US,75.00,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")